In [1]:
import matplotlib.pyplot as plt
from behavioural_benchmark.indicators import MemoisedIndicators
import numpy as np
from math import floor
import os
import random

To test and illustrate the indicators, let's construct examples where we know the answers. For each indicator we will create some data where we decide on the values of the indicators, and at the end we will assert that the calculated indicators match up with the expected ones.

## Regression indicators

For the regression based indicators, we can define graphs of two lines with set gradients. We can sample points along these lines, and save these to a file. Then later we can assert whether the regression indicators confirm the gradients we selected.

In [2]:
c = 100
x = np.array([j for j in range(0,100,2)])

def sample_points(m_1, m_2):
    y_1 = m_1 * x + c
    c_2 = y_1[floor(0.70 * len(y_1[y_1 >= 0]))]
    x_1 = x[y_1 >= c_2]
    y_1 = y_1[y_1 >= c_2]

    y_2 = m_2 * (x_1 + 1) + c_2
    x_2 = x_1 + max(x_1) + 1
    x_2 = x_2[y_2 >= 0]
    y_2 = y_2[y_2 >= 0]
    
    return np.concat([x_1, x_2]), np.concat([y_1, y_2]), max(x_1), c_2

def save_to_file(xf, yf, filename):
    data = np.column_stack((xf, yf))
    np.savetxt(f"example_data/{filename}.csv", data, delimiter=",", header=f"iteration,{filename}", comments="", fmt="%.4f")

# diversity
DRoC_A = -2
DRoC_B = -0.9
diversity_x, diversity_y, ERT_Diversity, Critical_Diversity = sample_points(DRoC_A, DRoC_B)
save_to_file(diversity_x, diversity_y, "diversity")

# fitness
FRoC_A = -1.8
FRoC_B = -1.0
fitness_x, fitness_y, ERT_Fitness, Critical_Fitness = sample_points(FRoC_A, FRoC_B)
save_to_file(fitness_x, fitness_y, "fitness")

# distance
SRoC_A = -2.05
SRoC_B = -0.75
distance_x, distance_y, ERT_Separation, Critical_Separation = sample_points(SRoC_A, SRoC_B)
save_to_file(distance_x, distance_y, "distance")

# Mobility
MRoC_A = -1.05
MRoC_B = -1.0
mobility_x, mobility_y, ERT_Mobility, Critical_Mobility = sample_points(MRoC_A, MRoC_B)
save_to_file(mobility_x, mobility_y, "mobility")

For the network based indicators, we need to construct interaction data, and trajectory data.

## Trajectory indicators

For the trajectory data, we can decide on a number of nodes and shared nodes. We can then simulate trajectories between these nodes, making sure that every node is visited at least once. We can create two runs, with shared nodes appearing in both runs. We can arbitrarily set the fitness of each node equal to 0, as it's not required for the indicators we are testing.

In [3]:
# STN
shared_nodes = list(range(20, 30))
nodes = {
    0: list(range(20)) + shared_nodes,
    1: shared_nodes + list(range(30, 30+20))
}
nbest = 5
best_nodes = random.sample(nodes[0] + nodes[1], nbest)
ntotal = len(np.unique(nodes[0] + nodes[1]))
nshared = len(shared_nodes)
best_strength = 0

file  = "example_data/stn.csv"
if os.path.exists(file):
    os.remove(file)
with open(file, "w") as f:
    f.write("Run,Fitness1,Solution1,Fitness2,Solution2\n")
    for run in [0,1]:
        current_position = dict(zip(range(10), random.sample(nodes[run], 10)))
        
        # Create a set of nodes to ensure each one is selected at least once
        required_nodes = set(nodes[run])
        selected_nodes = set(current_position.values())  # Keep track of selected nodes
        for _step in range(10):
            for individual in range(10):
                # Ensure each node is selected at least once
                if len(selected_nodes) < len(required_nodes):
                    # Pick a node from the unselected ones
                    next_position = random.choice(list(required_nodes - selected_nodes))
                    selected_nodes.add(next_position)
                else:
                    # Once all nodes have been selected at least once, continue with random choice
                    next_position = random.choice(nodes[run])
                if current_position[individual] in best_nodes:
                    current_fitness = 0
                else:
                    current_fitness = 1
                if next_position in best_nodes:
                    best_strength += 1
                    next_fitness = 0
                else:
                    next_fitness = 1
                f.write(f"{run},{current_fitness},{current_position[individual]},{next_fitness},{next_position}\n")
                current_position[individual] = next_position
best_strength /= 2  # number of runs

## Interaction  indicators

For the interaction data, we can create a small network of 5 individuals, resulting in a 5 x 5 grid. We can choose an IDRoC, and then model the interactions to result in the chosen IDRoC. We can also make sure the solution node has a weight equal to some chosen ISS.

In [4]:
# IN
MID = (1 + 2 + 3 + 5) / 4
MGC = (5 + 3 + 2 + 1) / 4 / 5  #  dividing by 5 normalises this, which is how it appears in their paper 
solution_index = 3
SNID = 30 / (2 * 10)

# let number of subgraphs be of 1, 2, 3, 5 (y)
# let size of subgraphs be 5, 3, 2, 1
# with maximum edge_weight removed equal to 0.0, 0.25, 0.5, 1.0 (x)
# (5 - 1) / (1.0 - 0.0) = 4

arr = np.array([
    # 0   1   2   3   4
    [ 0, 20,  0,  0,  0], # 0
    [20,  0,  5,  0,  0], # 1
    [ 0,  5,  0, 10,  0], # 2
    [ 0,  0, 10,  0, 20], # 3
    [ 0,  0,  0, 20,  0]  # 4
])

zeroes = np.zeros(shape=arr.shape)

file  = "example_data/interaction_network.txt"
if os.path.exists(file):
    os.remove(file)
with open(file, "w") as f:
    f.write(f"ig:#0 {' '.join(map(str, zeroes.flatten()))}\n")
    f.write(f"ig:#1 {' '.join(map(str, arr.flatten()))}\n")

## ENES, EXPLORE%, and INFEASIBLE%

Finally, only the "odd one out" indicators are left. These are ENES, EXPLORE%, and INFEASIBLE%. For INFEASIBLE% we need to write to file, the others are simple enough to verify.

In [5]:
f_percent = random.choices(population=range(0,100) ,k=10)
f_data = np.column_stack((range(0,10), f_percent))
np.savetxt(f"example_data/f_percent.csv", f_data, delimiter=",", header=f"iteration,f_percent", comments="", fmt="%.4f")
INFEASIBLE_Percent = sum(f_percent)/len(f_percent)

total_iterations = 10
iterations_before_target = 9 # this is a lie, doesn't matter
fitness_evaluations_before_target = iterations_before_target * 5 
global_best_fitness = 0
ENES = fitness_evaluations_before_target / iterations_before_target
EXPLORE_percent = np.mean(diversity_y / max(diversity_y) * 100)

file  = "example_data/metadata.json"
if os.path.exists(file):
    os.remove(file)
with open(file, "w") as f:
    f.write("{" + f'"total_iterations": {total_iterations}, "iterations_before_target":{iterations_before_target}, "fitness_evaluations_before_target": {fitness_evaluations_before_target}, "global_best_fitness": {global_best_fitness}, "solution_index": {solution_index}' + "}")

## Tests

In [6]:
k = MemoisedIndicators(path="example_data/")

def close_enough(expected, real):
    assert round(expected, 4) == round(real,4)

In [7]:
close_enough(DRoC_A, k.get_DRoC_A())
close_enough(DRoC_B, k.get_DRoC_B())
close_enough(ERT_Diversity, k.get_ERT_Diversity())
close_enough(Critical_Diversity, k.get_Critical_Diversity())

close_enough(FRoC_A, k.get_FRoC_A())
close_enough(FRoC_B, k.get_FRoC_B())
close_enough(ERT_Fitness, k.get_ERT_Fitness())
close_enough(Critical_Fitness, k.get_Critical_Fitness())

close_enough(SRoC_A, k.get_SRoC_A())
close_enough(SRoC_B, k.get_SRoC_B())
close_enough(ERT_Separation, k.get_ERT_Separation())
close_enough(Critical_Separation, k.get_Critical_Separation())

close_enough(MRoC_A, k.get_MRoC_A())
close_enough(MRoC_B, k.get_MRoC_B())
close_enough(ERT_Mobility, k.get_ERT_Mobility())
close_enough(Critical_Mobility, k.get_Critical_Mobility())

In [8]:
close_enough(ntotal, k.get_ntotal())
close_enough(nbest, k.get_nbest())
close_enough(nshared, k.get_nshared())
close_enough(best_strength, k.get_best_strength())

In [9]:
close_enough(MID, k.get_MID())
close_enough(MGC, k.get_MGC())
close_enough(SNID, k.get_SNID())

In [10]:
close_enough(ENES, k.get_ENES())
close_enough(EXPLORE_percent, k.get_EXPLORE_Percent())
close_enough(INFEASIBLE_Percent, k.get_INFEASIBLE_Percent())